# **Análise de Dados e Visualização para Tomada de Decisões em Publicidade**

### **Teste Técnico - Analista de Dados Jr - RBS**

#### Bruno Di Franco Albuquerque

## **Descrição do Teste:**

Neste teste, os candidatos deverão realizar uma análise de dados simples relacionada a campanhas publicitárias e criar visualizações claras para ajudar na tomada de decisões de negócios. Eles terão acesso a um conjunto de dados fictício que contém informações sobre campanhas publicitárias e desempenho das mesmas. 

Vamos começar importando as bibliotecas necessárias para a análise:

In [1]:
import pandas as pd

Agora podemos ler os dados e visualizar uma amostra dos mesmos:

In [2]:
df = pd.read_csv("Base de dados - Case Analista Jr - RBS Performance.csv", sep=";")
df.sample(5)

,advertiserName,date,creativeType,deviceCategoryName,totalLineItemLevelImpressions,totalLineItemLevelClicks,totalLineItemLevelCtr,clickThroughConversions,viewThroughConversions
777,Cliente 1,12/06/2023,Other,Desktop,5908,42,"0,00711",41,401
692,Cliente 1,10/06/2023,DoubleClick Rich Media,Smartphone,7121,106,"0,01489",85,314
6007,Cliente 1,18/08/2023,Other,SmartTV,39842,127,"0,00319",29,237
1511,Cliente 1,21/06/2023,Video creative sets,SmartTV,21807,319,"0,01463",75,184
653,Cliente 1,10/06/2023,Other,Tablet,29478,487,"0,01652",197,323


## **Tarefas:**

### 1. **Exploração de Dados**

#### 1.1. **Examinar a estrutura do conjunto de dados (quantidade de linhas e colunas)**

In [3]:
qtd_linhas = df.shape[0]
qtd_colunas = df.shape[1]

print(f"A base de dados possui {qtd_linhas} linhas e {qtd_colunas} colunas.")

A base de dados possui 7000 linhas e 9 colunas.


#### 1.2. **Identificar as principais variáveis presentes**

In [4]:
df.dtypes

advertiserName                   object
date                             object
creativeType                     object
deviceCategoryName               object
totalLineItemLevelImpressions     int64
totalLineItemLevelClicks          int64
totalLineItemLevelCtr            object
clickThroughConversions           int64
viewThroughConversions            int64
dtype: object

Ao analisar os tipos de cada variável, identificam-se duas variáveis que podem ser convertidas para tipos mais apropriados em relação ao seu significado:

- `date`, que é a data, porém está como `object`.
- `totalLineItemLevelCtr`, que é a Taxa de Cliques (click through rate), está como `object`, mas deveria ser um `float`.

In [5]:
df["date"] = pd.to_datetime(df["date"], format="%d/%m/%Y")
df["totalLineItemLevelCtr"] = df["totalLineItemLevelCtr"].str.replace(",", ".").astype(float)

Agora o tipo e formato dessas duas variáveis estão mais apropriados.

In [11]:
print(df[["date", "totalLineItemLevelCtr"]].dtypes)
df[["date", "totalLineItemLevelCtr"]].sample(5)

date                     datetime64[ns]
totalLineItemLevelCtr           float64
dtype: object


,date,totalLineItemLevelCtr
1290,2023-06-19,0.00593
3043,2023-07-11,0.01245
2396,2023-07-03,0.00563
4957,2023-08-04,0.03083
1005,2023-06-15,0.01654


Agora podemos seguir para a análise das "principais variáveis presentes":

As variáveis presentes podem ser classificadas em 3 grupos:

Primeiro grupo:

- Esse primeiro grupo representa as variáveis categóricas presentes na base de dados.

| **Nome da Variável** | **Significado** | **Tipo da Variável** |
|----------------------|-----------------|----------------------|
|  advertiserName        |   Nome do Cliente              |    objeto, string                  |
| creativeType                     | Tipo do Criativo                | objeto, string          |
|   deviceCategoryName       |  Categoria do dispositivo                  |  objeto, string    |

Segundo grupo:

- No segundo grupo temos apenas uma variável, que é a data, sendo, portanto, uma variável temporal, que pode ser muito útil para visualizações de séries temporais.

| **Nome da Variável** | **Significado** | **Tipo da Variável** |
|----------------------|-----------------|----------------------|
|  date        |   Data de exibição do anúncio              |    datetime, variável temporal                |

Terceiro grupo:

- Esse último grupo representa as variáveis numéricas, podendo ser encaradas como KPI's para publicidade, como indicadores de conversão, taxa de cliques, etc. **Portanto, se tivermos que elencar as "principais variáveis presentes" seriam exatamente as desse terceiro grupo.**

| **Nome da Variável** | **Significado** | **Tipo da Variável** |
|----------------------|-----------------|----------------------|
|  totalLineItemLevelImpressions         |    Impressões totais             |    numérica, integer              |
| totalLineItemLevelClicks               | Cliques totais             | numérica, integer        |
|   totalLineItemLevelCTR                |  Taxa de Cliques (click through rate)      |  numérica, float    |
| clickThroughConversions                | Conversões diretas (oriundas de clique)        | numérica, integer        |
|   viewThroughConversions               |  Conversões indiretas (não relacionadas ao último clique)    |  numérica, integer   |

### 2. **Análise Descritiva**

#### 2.1. **Qual é a média de conversões por tipo de criativo?**

In [12]:
df_aux = df[["creativeType","clickThroughConversions", "viewThroughConversions"]]

In [13]:
df_aux

,creativeType,clickThroughConversions,viewThroughConversions
0,Custom,204,369
1,Custom template,21,368
2,Programmatic,97,327
3,DoubleClick Rich Media,22,256
4,Video creative sets,325,63
...,...,...,...
6995,Custom,41,165
6996,Third-party,195,143
6997,Third-party,58,323
6998,Video creative sets,122,6


In [16]:
df

,advertiserName,date,creativeType,deviceCategoryName,totalLineItemLevelImpressions,totalLineItemLevelClicks,totalLineItemLevelCtr,clickThroughConversions,viewThroughConversions
0,Cliente 1,02/06/2023,Custom,SmartTV,11983,349,"0,02912",204,369
1,Cliente 1,02/06/2023,Custom template,Smartphone,12507,112,"0,00895",21,368
2,Cliente 1,02/06/2023,Programmatic,Tablet,29909,488,"0,01632",97,327
3,Cliente 1,02/06/2023,DoubleClick Rich Media,SmartTV,39189,172,"0,00439",22,256
4,Cliente 1,02/06/2023,Video creative sets,Desktop,41458,362,"0,00873",325,63
...,...,...,...,...,...,...,...,...,...
6995,Cliente 1,30/08/2023,Custom,Desktop,41794,113,"0,00270",41,165
6996,Cliente 1,30/08/2023,Third-party,Desktop,2633,459,"0,17433",195,143
6997,Cliente 1,30/08/2023,Third-party,SmartTV,34281,241,"0,00703",58,323
6998,Cliente 1,30/08/2023,Video creative sets,SmartTV,35835,463,"0,01292",122,6
